In [1]:
import mechanize
import http.cookiejar
from bs4 import BeautifulSoup
import re

import pandas as pd
import time

In [2]:
# Browser
br = mechanize.Browser()

# Cookie Jar
cj = http.cookiejar.LWPCookieJar()
br.set_cookiejar(cj)

# Browser options
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

br.addheaders = [('User-agent', 'Chrome')]

# URL to load
url = 'https://mn.wiktionary.org/w/index.php?title=Ангилал:Монголоор&from=А'

# Loaded html
html = br.open(url).read()

# Read html with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Find and print title to check we have loaded correctly
print(soup.find('h1').string.strip())

Ангилал:Монголоор


In [4]:
# Letters of Mongolian alphabet
tolgoi = ["А", "Б", "В", "Г", "Д", "Е", "Ё", "Ж",
          "З", "И", "Й", "К", "Л", "М", "Н", "О",
          "Ө", "П", "Р", "С", "Т", "У", "Ү", "Ф", 
          "Х", "Ц", "Ч", "Ш", "Щ", "Э", "Ю", "Я",]

# List to contain words
words = []

for letter in tolgoi:
    
    # URL to load
    url = 'https://mn.wiktionary.org/w/index.php?title=Ангилал:Монголоор&from=' + letter

    # Loaded html
    html = br.open(url).read()

    # Read html with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    
    for element in soup.find('div', id='mw-pages').contents[9].contents[0].find_all('a'):
        words1.append(element.string.strip())

words = pd.Series(words1).str.lower().drop_duplicates()
words

0                                  а
1                                 аа
2                                аав
3                           аавархах
4                             аавсаг
                    ...             
6393                           яйшин
6394                              ял
6396     ял хүндрүүлэх нөхцөл байдал
6397    ял хөнгөрүүлэх нөхцөл байдал
6398             ял шийтгэлгүй болох
Length: 5523, dtype: object

In [21]:
words1[268]

'баатар'

Edit to allow for saving multiple example sentences

In [81]:
df = pd.DataFrame({'Mongolian': words1[:3318], 'English': translation})
df = df[df['English'] != '']
df.to_excel('bolor_toli_test.xlsx', index=False)
df = pd.DataFrame({'Mongolian': examples_mn, 'English': examples_en})
df = df[df['Mongolian'] != '']
df.to_excel('bolor_toli_test_examples.xlsx', index=False)

# Sandbox

In [12]:
# The site we will navigate into, handling it's session
br.open('http://www.bolor-toli.com/')

# List for storing translations
translation = []
examples_mn = []
examples_en = []
examples = []

for word in ['тухай']:
    
    # Select the second (index one) form
    br.select_form(nr=1)

    # Search
    br.form['search'] = word
    br.submit()

    # html to read
    html = br.response().read()
    
    # Read html with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    
    if soup.find(class_='link-string') != None:
        tr = soup.find(class_='link-string').string
        translation.append(tr)
        
        # Find example sentence if it exists
        examples = soup.find_all(class_='blank-td')
        
        if examples != []:
            mn = ''
            en = ''
            ex = ''
            for example in examples:
                mn = example.next_sibling.next_sibling.string.strip()
                en = example.next_sibling.next_sibling.next_sibling.next_sibling.string.strip()
                ex += '<p>' + mn + ': ' + en + '</p>'
                print(ex)
            examples_mn.append(mn)
            examples_en.append(en)
            examples.append(ex)
        else:
            examples_mn.append("")
            examples_en.append("")
            examples.append("")
        
        print(tr)
    else:
        translation.append("")
        examples_mn.append("")
        examples_en.append("")
        examples.append("")
        print('Not available')

translation = pd.Series(translation).str.lower()
examples_mn = pd.Series(examples_mn).str.lower()
examples_en = pd.Series(examples_en).str.lower()
examples = pd.Series(examples)

<div>цэцгийн тухай ном: a book about flowers</div>
<div>цэцгийн тухай ном: a book about flowers</div><div>ирээдүйн тухай тодорхой бус санаа бодол: nebulous ideas about the future</div>
<div>цэцгийн тухай ном: a book about flowers</div><div>ирээдүйн тухай тодорхой бус санаа бодол: nebulous ideas about the future</div><div>ямар нэг юмны тухай шийдвэр: a decision on sth.</div>
<div>цэцгийн тухай ном: a book about flowers</div><div>ирээдүйн тухай тодорхой бус санаа бодол: nebulous ideas about the future</div><div>ямар нэг юмны тухай шийдвэр: a decision on sth.</div><div>ямар нэг юмны тухай шийдвэр: a decision on smth.</div>
<div>цэцгийн тухай ном: a book about flowers</div><div>ирээдүйн тухай тодорхой бус санаа бодол: nebulous ideas about the future</div><div>ямар нэг юмны тухай шийдвэр: a decision on sth.</div><div>ямар нэг юмны тухай шийдвэр: a decision on smth.</div><div>цөмийн туршилтыг хориглох тухай гэрээ: treaty on ban of nuclear test</div>
<div>цэцгийн тухай ном: a book about flowe

In [8]:
for element in soup.find_all(class_='blank-td'):
    print(element.next_sibling.next_sibling.string.strip())

цэцгийн тухай ном
ирээдүйн тухай тодорхой бус санаа бодол
ямар нэг юмны тухай шийдвэр
ямар нэг юмны тухай шийдвэр
цөмийн туршилтыг хориглох тухай гэрээ
төлбөр хойшлуулах тухай хүсэлт
нөхөрлөлийн тухай хэлэлцээр
ослын тухай мэдээ
